# Power perp mechanism, arbitrage, and use cases

**Joseph Clark, Andrew Leone, and Zubin Koticha -  opyn finance**


<br>

*(content warning: equations, option greeks)*

<br>

Most derivative contracts have fixed expiries. To get a continuous exposure to an asset price, investors must "roll" contracts, selling out of an expiring contract and buying a longer dated contract (or vice versa).

Rolling contracts costs time and money, and is generally bad UX.

The idea of perpetual contracts (perps) is to give a financial exposure similar to rolling fixed expiry contracts in a single contract that never expires. Owing to their ease of use, they are more widely used in cryptocurrency than futures.

Power perpetuals are a new generalization of perpetual swaps and can replicate stablecoins, futures, options, and more.

Squeeth is a tokenized quadratic power perpetual.

tl/dr;

- A 0-perp is a stablecoin
- A 1-perp is a future
- Any p-perp that is not 0 or 1 is a volatility oracle
- Power perps can be traded against fixed-expiry power futures 
- A 2-perp (squeeth) is an excellent hedge for options and constant function market makers such as uniswap and curve
- A 0.5 perp (sqrth) is a perfect hedge for a uniswap LP position and it's coming next!

 
## Power futures and Power Perpetuals

Power futures are derivatives paying some power $p$ of an asset price $S(T)$ at a future date $T$. A linear future ($p=1$) pays the asset price at expiry. A quadratic future ($p=2)$ pays the square of the price, a sqare root future ($p=1/2$) pays the square root of the price, and so forth. 

Power futures are priced in same way as options, by calculating the expected value under an appropriate probabilty distribution.  If the price is lognormal with constant volatility, a contract paying $S(T)^p$ at time $T$ should price at time $t$ at

$F^p(t,T) = S(t)^p e^{((p-1)(r+ \frac{p\sigma^2}{2}) -pq )(T-t)}$

(see <a href="https://colab.research.google.com/drive/1HTM_2j0jmda9tzN_uskBPz9Rpma8Lp3C" title="Power Perpetuals">here</a> for a development)

Where $r$ is the numeriare interest rate, $q$ is the asset yield, and $\sigma$ is the volatiltiy.

Power perpetuals track the exposure of rolling the underlying power future by paying or recieving a regular funding rate.

A zero perpetual (0-perp) tracks a price of $S(T)^0=1$. The funding rate should be close to $r$. In other words, holding the perpetual is similar or rolling a series of short term bonds.

DAI can be thought of as a 0-perp.

A one-half perpetual (1/2-perp) tracks a price of $S(T)^{1/2}$. The funding rate should be close to $\sigma^2/8$. Holding the perpetual is similar to rolling a series of fixed-expiry square root futures.

A Uniswap LP token can be thought of as a 1/2-perp.

A linear perpetual (1-perp) tracks the asset price $S(T)^1$ and has a funding rate close to an expiring futures contract. Holding the linear perpetual is similar to rolling a series of fixed-expiry linear futures. 

FTX or dYdX perp-swaps can be thought of as 1-perps.

A quadratic perpetual (2-perp) tracks the asset price $S(T)^2$ and  has a funding rate close to an expiring quadratic future. Holding the perpetual is similar to rolling a series of fixed-expiry quadratic futures. 

Squeeth (squared ETH) is the first 2-perp.



## Perp mechanism with continuous in-kind funding (MakerDAO style)

One implementation of a perpetual contract is the following:

- Contract mints one unit of a p-perp at time $t$.
- The contract records a debt of $1 NF(t) * S(t)^p$ where $S(t)$ is the asset price and $NF(t)$ is a normalization factor that accumulates a continuous funding rate.
- The contract can be repaid at any time by returning 1 unit of the power perpetual or by repaying the whole debt value (and any outstanding funding).
- The contract requires collateral be worth more than the debt. If the value of collateral falls below some multiple of the debt value, the contract liquidates the collateral to repay the loan.

(Note: this is not precisely *Opyn's* implementation)

This mechanism creates a fungible asset that tracks $S(t)^p$ and avoids machinery for paying or receiving cash funding. The perpetual should trade close to its debt value (adjusted for funding) in an external market. 

The funding rates embedded in the price should reflect the cost of carry for the asset so an investor is indifferent between holding the perpetual and holding a series of short expiry $p$-futures.

### Constructing the normalization factor

The normalization factor can be constructed as the cumulative impact of carry (net funding) on the $p$-future:

$NF(t) = \exp(\int_0^t c_p(t) dt)$

Where $c_p(t)$ is the continuously compounded funding rate for the p-perp at time $t$.

This modifies the debt of the power perp-minter and so is implicitly a payment between longs and shorts. 

### Theoretical value of funding payment under log-normal price dynamics

The carry on power futures for lognormal prices (under BSM assumptions) measures the relative rate of return between the future and the underlying index $S(t)^p$. This is the cost of holding the position relative to:

$c_p(t) = -\frac{\partial ln \frac{F}{S(t)^p}}{\partial \tau} = -(p-1)(r+ \frac{p\sigma^2}{2}) + pq$

Where $\tau = T-t$ is time to expiry.

For a zero perpetual ($p=0$) the funding rate $c_0(t) = r$. Say I mint a 0-perp now. If the annual interest rate is 10\% the normalization factor after one year will be $e^{0.1*1}$ so it will cost 1.11 to close the position. This is equivalent to borrowing one unit of the numeraire. The perp itself will appreciate at a constant rate. This is a collateralized zero coupon bond -- the original stablecoin.

For a linear perpetual ($p=1$) the funding rate is $c_1(t)=q$. Say I mint a 1-perp now. If the asset yield is 5\% the normalization factor after a year will be $e^{0.05*1}=1.05$. If the price has moved to 90 it will cost $1.05*90 = 94.5$ to repay. The perp minter is short the asset. This is a perpetual future.

For a quadratic future the funding rate is $c_2(t) = -(r+ \sigma^2) + 2q$. Say I mint a 2-perpetual now. If rates are 10\% and volatility is 50\% the normalization ratio after one year will be $exp(r+ \sigma^2) = exp(-(0.1 + 0.5^2)) = 0.70$. If the price has moved to 90 it will cost $90^2 * 0.70$ to repay. This is a short quadratic perpetual. We call it squeeth.


![0-perp](https://drive.google.com/uc?export=view&id=1ifmvIohJj66SOKYw5o1CSJOx1Z9iqeE8)
![1-perp](https://drive.google.com/uc?export=view&id=1JmBoDW1Qcz8foEMxPlGtYACpbw0cFZ-F)
![2-perp](https://drive.google.com/uc?export=view&id=1ckBge6LE-GmJGdT3cTuiErOIxYsW7TYH)


### Calibrating funding to traded value of perp

We know what the funding rate *should* be. To get the funding rate to match, we might* set 
 
$c_p(t) = - \frac{1}{f} \log \frac{M_p(t)/NF(t)}{S(t)^p}$

where $M_p(t)$ is the prevailing market price (mark) of the p-perp, $S^p(t)$ is the index we want to track, and $f$ is a funding period.

This means that the contract will pay or receive a funding rate equal to the percentage difference between the de-normalized mark and index, scaled by a constant representing the funding period.

If the perp trades trades 1\% away from from the index value for a day and $f=1/365$, the contract will pay 1% funding for the day.

The market price will adjust to make this funding equal the carry on a fixed expiry power contract. If it is lower, a trader will buy the perpetual (implicitly paying the unnaturally low funding), and short the fixed expiry future. If it is higher, a trader will mint the perpetual, sell at the unnaturally high price, and buy the future. 

#### Example: perp cheap arb 

We have a 2-perp market with the mechanism described. At $t=0$:

- Asset price is 100
- Normalization factor is 1
- Prevailing market price for the 2-perp is 10008
- The prevailing market price for the 1 day quadratic contract is 10009

Alice the arb-trader immediately calculates that the fixed expiry contract is pricing 57% implied volatility and the perp is pricing 54% implied volatility.

Alice buys the perp and shorts the fixed expiry future.

At $t=1/365$ (+ one day):

- Asset price is still 100
- Normalization factor is 0.9992 ( = exp(-ln(10008/10000)) )
- New market price for the 2-perp is 10000 (pricing the same implied volatility as before)
- The expiring 1 day quadratic contract is 10000 

Alice loses 8 on the perp but makes 9 on the future, pocketing 1. Note that this would be profitable regardless of the the asset price moves.

For simplicity, we assume the implied volatility is constant. We also assume the existence of a fixed expiry quadratic contract, though this is less restrictive since a close replica can be constructed with a portfolio of European options.

This is also not a perfect hedge, since we are hedging volatilty with different durations. The exact hedge is discussed <a href="https://www.paradigm.xyz/2021/08/power-perpetuals/" title="Power Perpetuals">here</a> and <a href="https://colab.research.google.com/drive/1HTM_2j0jmda9tzN_uskBPz9Rpma8Lp3C" title="More than you ever wanted to know about power perpetuals">here</a>. 

#### Example: perp rich arb 

We have a 2-perp market with the mechanism described. At $t=0$:

- Asset price is 100
- Normalization factor is 1
- Prevailing market price for the 2-perp is 10011
- The prevailing market price for the 1 year quadratic contract is 10009

Alice the arb-trader immediately calucates that the fixed expiry contract is pricing 60% volatility and the perp is pricing 63% volatility.

Alice mints a perp and immediately sells it, and buys the fixed expiry future.


At $t=1/365$ (+ one day):

- Asset price is 100
- Normalization factor is 0.9989 ( = exp(-ln(10011/10000)) )
- New market price for the 2-perp is 10000 (pricing the same implied volatility as before)
- The expiring quadratic contract is 10000

Alice buys back the perp and redeems her collateral. She makes 11 on the perp and loses 9 on the future, pocketing 2.

This example demonstrates a nice property of perpetuals - the price of a perpetual with one day funding period is approximately to the one day fixed expiry contract. This makes it very easy to determine the correct market price of the perp by looking the price of the fixed expiry contract implied by option prices.


## A properly functioning 2-perp market

Traders busy themselves calculating the funding rate implied by the current market price, which in the case of a 2-perp is the implied variance, and check this against the price implied by a related fixed expiry p-contract or the options. If it is higher or lower it can be traded against the fixed expiry contract (or its replicating portfolio of options). 

These adjustment trades are not quite arbitrages, since there is no guaranteee that the traded price will converge, but there are very good incentives for the price to be correct. Again, the exact portfolio is <a href="https://www.paradigm.xyz/2021/08/power-perpetuals/" title="Power Perpetuals">here</a> and <a href="https://colab.research.google.com/drive/1HTM_2j0jmda9tzN_uskBPz9Rpma8Lp3C" title="More than you ever wanted to know about power perpetuals">here</a>.

## 2-perps as option gamma hedges

An important use case for the 2-perp (squeeth) is as a hedge for options. Because it is strike-less and expiry-less, it can be used as a hedging instrument for *all* options.


An option can be usefully decomposed into three primary risks:

- **delta risk** - exposure to changes in the asset price 
- **gamma risk** - exposure to squared changes in asset price
- **vega risk** - exposure to changes in volatility expectation

The figure below shows the pnl for a call option based on changes in the underlying price.


![Call attribution](https://drive.google.com/uc?export=view&id=1tpHezUHjS57YEafStFqC4G4uCIJ8BFLQ)

Hedging the delta risk with a future reduces risk (orange line below) leaves a large exposure to squared price changes (gamma). Adding a quadratic hedge substantially reduces the risk (green line below). 

An available quadratic hedging instrument (squeeth) reduces the risk of market making options. This improves price discovery and liquidity in the options market.

![Call hedge](https://drive.google.com/uc?export=view&id=1Y1cxlB1Ui7YwqJwIyfSGnPa3ArNwWhIJ)




### 2-perps as AMM (uniswap) LP hedges

Another important use case for 2-perp is as a hedge for uniswap and similar constant function market makers (CFMMs). The hedge is very similar to options.

A (unbounded) uniswap LP has a payoff

$2\sqrt{k S(t)}$ 

where $S(t)$ is the exchange rate between the two pool tokens and $k$ is the product of the two pool balances.

Much like an option, this risk can be decomposed into 

- **delta risk** - exposure to changes in the asset price 
- **gamma risk** - exposure to squared changes in asset price

The figure below shows the pnl for a uniswap LP based on changes in the underlying price.

![AMM attribution](https://drive.google.com/uc?export=view&id=16a-krpYL9wNLmeTiGe5BfA8tUH-2nYYL)

Hedging the delta risk with a future reduces risk (orange line below) leaves a large exposure to squared price changes (gamma). Adding a quadratic hedge substantially reduces the risk (green line below). 


![AMM attribution](https://drive.google.com/uc?export=view&id=1uX6L1WDC8j_tw8zoC06bnKcY73z9YtyV)

An available quadratic hedging instrument (squeeth) reduces the risk of providing liquidity in CFMM pools like uniswap because the gamma risk (sometimes called "impermanent loss") can be hedged very closely.

This improves liquidity provision uniswap pools where liquidity will increase to the point at which fee revenue is offset by the expected cost of holding the 2-perp.

A 2-perp is a good hedge for imperment loss, but a better hedge is a 0.5 perp since this perfectly matches the risk of the LP token. We call this sqrth and it's coming next.



(*) The actual calculation is similar but a bit less intuitive.





## Useful resources



*Power Perpetuals*: https://www.paradigm.xyz/2021/08/power-perpetuals/

*More Than You Ever Wanted To Know About Power Perpetuals*: https://colab.research.google.com/drive/1HTM_2j0jmda9tzN_uskBPz9Rpma8Lp3C


*Replicating Monotonic Payoffs Without Collateral*:
https://stanford.edu/~guillean/papers/cfmm-monotone.pdf

*The Replicating Portfolio of a Constant Product Market with Bounded Liquidity*:
https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3898384


Acknowledgements: <a href="https://twitter.com/llllvvuu" title="llllvvuu">llllvvuu</a>, <a href="https://twitter.com/_Dave__White_
" title="Dave White">Dave White</a> , <a href="https://twitter.com/danrobinson
" title="Dan Robinson">Dan Robinson</a> .